<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Norway_Oslo_Meteorological_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
client_id = '95b97914-f898-42d6-b1de-b1557d61357d'

# 1. CORRECTED TASKS
# We revert 'wind_speed' to its simple name, which is compatible with this station.
tasks = [
    # Solar: 1-minute average
    {"station": "SN18700", "element": "mean(surface_downwelling_shortwave_flux_in_air PT1M)", "name": "Global_Radiation_W/m2"},

    # Temp: Generic name (Automatic resolution)
    {"station": "SN18700", "element": "air_temperature", "name": "Temperature_C"},

    # Wind: Generic name (Automatic resolution - usually 1 min or 10 min for Blindern)
    {"station": "SN18700", "element": "wind_speed", "name": "Wind_Speed_ms"}
]

months = [
    ('2024-01-01', '2024-02-01'), ('2024-02-01', '2024-03-01'),
    ('2024-03-01', '2024-04-01'), ('2024-04-01', '2024-05-01'),
    ('2024-05-01', '2024-06-01'), ('2024-06-01', '2024-07-01'),
    ('2024-07-01', '2024-08-01'), ('2024-08-01', '2024-09-01'),
    ('2024-09-01', '2024-10-01'), ('2024-10-01', '2024-11-01'),
    ('2024-11-01', '2024-12-01'), ('2024-12-01', '2025-01-01')
]

dfs = []
print("Fetching 2024 Data from Oslo Blindern (SN18700)...")

for task in tasks:
    print(f"   > Requesting {task['name']}...")
    all_rows = []

    for start, end in months:
        params = {
            'sources': task['station'],
            'elements': task['element'],
            'referencetime': f"{start}/{end}",
        }

        try:
            r = requests.get('https://frost.met.no/observations/v0.jsonld', params, auth=(client_id, ''))

            if r.status_code == 200:
                data = r.json().get('data', [])
                for item in data:
                    val = None
                    for obs in item['observations']:
                        if task['element'] == obs['elementId']:
                            val = obs['value']
                            break

                    if val is not None:
                        all_rows.append({'Time': item['referenceTime'], task['name']: val})
            else:
                if r.status_code == 412:
                     print(f"     [!] API Error: Element '{task['element']}' not found.")
                     # Don't break immediately, let it try the next month just in case,
                     # though usually 412 means it's invalid for the whole station.

        except Exception as e:
            print(f"     Error: {e}")
        time.sleep(0.1)

    if all_rows:
        df = pd.DataFrame(all_rows)
        df['Time'] = pd.to_datetime(df['Time'])
        df = df.drop_duplicates(subset='Time')
        df = df.set_index('Time').sort_index()
        dfs.append(df)
        print(f"     Success: Got {len(df)} rows.")
    else:
        print(f"     WARNING: No data retrieved for {task['name']}.")

# --- MERGE & SAVE ---
if dfs:
    print("Merging data...")
    # 'outer' join keeps all timestamps from all variables
    df_final = pd.concat(dfs, axis=1, join='outer')

    # Interpolate small gaps
    df_final = df_final.interpolate(method='linear', limit=15)

    df_final = df_final.reset_index()
    df_final['Date'] = df_final['Time'].dt.strftime('%Y-%m-%d')
    df_final['Hour'] = df_final['Time'].dt.strftime('%H:%M')

    cols_to_keep = ['Date', 'Hour', 'Temperature_C', 'Global_Radiation_W/m2', 'Wind_Speed_ms']

    # Filter columns safely
    final_cols = [c for c in cols_to_keep if c in df_final.columns]
    df_final = df_final[final_cols]

    print(df_final.head())

    filename = "Oslo_Blindern_2024_Full.csv"
    df_final.to_csv(filename, index=False)
    print(f"\nSuccess! Saved to {filename}")
else:
    print("Failed to build dataset.")

Fetching 2024 Data from Oslo Blindern (SN18700)...
   > Requesting Global_Radiation_W/m2...
     Success: Got 504203 rows.
   > Requesting Temperature_C...
     Success: Got 467067 rows.
   > Requesting Wind_Speed_ms...
     Success: Got 467095 rows.
Merging data...
         Date   Hour  Temperature_C  Global_Radiation_W/m2  Wind_Speed_ms
0  2024-01-01  00:00          -4.00                   -0.9           7.20
1  2024-01-01  00:01          -4.01                   -0.9           7.24
2  2024-01-01  00:02          -4.02                   -0.6           7.28
3  2024-01-01  00:03          -4.03                   -0.7           7.32
4  2024-01-01  00:04          -4.04                   -0.9           7.36

Success! Saved to Oslo_Blindern_2024_Full.csv
